# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is the SysML v2 Kerbal model written by Bjorn Cole

The textual SysML v2 model is the Kerbal model:

    package Kerbal {
        package 'Rocket Building' {
            import ScalarFunctions::*;

            part def Rocket {
                part stages : 'Rocket Stage' [1..5] {
                    // placing this here because previous stages only make sense in context of a full vehicle
                    ref 'Carried Stage' : 'Rocket Stage' [1];
                    attribute 'Payload Mass' : Real;
                    attribute 'Loaded Mass' : Real;
                    attribute 'Burnout Mass' : Real;
                    part 'Coupler to Carrying Stage' : Coupler [0..8] {
                        attribute 'Separation Force' : Real;
                    }
                }
            }
            item def Oxidizer;
            item def Fuel;
            item def 'Solid Propellant';

            // TODO: Something something part symmetry
            abstract part def 'Rocket Stage';

            // use stage types to enforce matching
            part def 'Liquid Stage' :> 'Rocket Stage' {
                part engines : 'Liquid Engine' [0..8];
                part tanks : 'Fuel Tank Section' [0..30];
                attribute 'Full Mass' : Real = sum(engines->collect p:'Kerbal Rocket Part' (p::Mass)) +
                                        sum(tanks->collect p:'Fuel Tank Section' (p::'Full Mass'));

                attribute 'Empty Mass' : Real;
            }
            part def 'Solid Stage' :> 'Rocket Stage' {
                part boosters : 'Solid Booster' [0..8];
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            part def 'Coupler' :> 'Kerbal Rocket Part';

            abstract part def 'Fuel Tank Section' :> 'Kerbal Rocket Part' {
                attribute 'Liquid Fuel' : Real;
                attribute 'Oxidizer' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            abstract part def 'Liquid Engine' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Thrust' : Real;
            }
            abstract part def 'Solid Booster' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
                attribute 'Thrust' : Real;
            }

            part def 'Pod' :> 'Kerbal Rocket Part'  {
                attribute Torque : Real;
            }
            part def 'Parachute' :> 'Kerbal Rocket Part';

            part def 'Kerbal Rocket Part' {
                attribute Mass : Real;
                attribute 'Max Temperature' : Real;
            }
        }
        package 'Parts Library' {
            import ScalarFunctions::*;
            part def 'FL-T200 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 1.125;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.125;
            }
            part def 'FL-T100 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 0.5625;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.0625;
            }
            part def 'Mk1 Command Pod' :> 'Rocket Building'::'Pod';
            part def 'LV-T45 "Swivel" Liquid Fuel Engine' :> 'Rocket Building'::'Liquid Engine' {
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Liquid Engine'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Liquid Engine'::'Thrust' = 167.97;
                attribute 'Mass' : Real :>> 'Rocket Building'::'Kerbal Rocket Part'::Mass = 1.50;
            }
            part def 'RT-5 "Flea" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 1.50;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.45;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 140.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::'Thrust' = 162.91;
            }
            part def 'RT-10 "Hammer" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 3.56;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.75;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::Thrust = 197.90;
            }
        }
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [ ]:
import os
from pathlib import Path

import networkx as nx

import pymbe.api as pm
from pymbe.client import SysML2Client
from pymbe.label import *
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.calc_dependencies import generate_execution_order, generate_parameter_signature_map
from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.local.stablization import build_stable_id_lookups

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [ ]:
helper_client = SysML2Client()

elements_data_path = Path("..") / "tests" / "fixtures" / "Kerbal.json"
assert elements_data_path.exists(), f"Could not find: '{elements_data_path}'"

helper_client._load_from_file(elements_data_path)

Create a graph representation of the model and load it into memory.

In [ ]:
lpg = SysML2LabeledPropertyGraph()
lpg.model = helper_client.model

This is just a helper to make abbreviations more legible.

In [ ]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

In [ ]:
name_to_id_lookup = build_stable_id_lookups(lpg)[1]
name_to_id_lookup

In [ ]:
ROCKET_BUILDING = "Model::Kerbal::Rocket Building::"
PARTS_LIBRARY = "Model::Kerbal::Parts Library::"

fts_full_mass = name_to_id_lookup[f'{ROCKET_BUILDING}Fuel Tank Section::Full Mass: Real <<AttributeUsage>>']
ft200_full_mass = name_to_id_lookup[f'{PARTS_LIBRARY}FL-T200 Fuel Tank::Full Mass: Real <<AttributeUsage>>']
ft100_full_mass = name_to_id_lookup[f'{PARTS_LIBRARY}FL-T100 Fuel Tank::Full Mass: Real <<AttributeUsage>>']
liquid_stage_full_mass = name_to_id_lookup[f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real <<AttributeUsage>>']
top_plus = name_to_id_lookup[f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real::+ (sum (collect (FRE.engines)), ' + 
                             'sum (collect (FRE.tanks))) => $result <<OperatorExpression>>'] # The '+' Expression under the Full Mass Attribute under Liquid Stage
tank_mass_sum_1 = name_to_id_lookup[f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real::+ (sum (collect (FRE.engines)),' + 
                                    ' sum (collect (FRE.tanks))) => $result::sum (collect (FRE.tanks)) => $result <<InvocationExpression>>'] # The 'sum' Expression that sums Full Mass of tanks
full_mass_dot = '86a423f7-18b6-4a86-ac52-2f69302121ab' # Expression that scopes Full Mass attribute in FeatureReferenceExpression
fre_1 = name_to_id_lookup[f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real::+ (sum (collect (FRE.engines)),'
                  f' sum (collect (FRE.tanks))) => $result::sum (collect (FRE.tanks)) => $result::collect (FRE.tanks) => $result'
                  f'::FRE.Full Mass (p) => $result::FRE.Full Mass <<FeatureReferenceExpression>>'] # Expression to point to the instances of Full Mass
fre_1_result = name_to_id_lookup[f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real::+ (sum (collect (FRE.engines)),'
                  f' sum (collect (FRE.tanks))) => $result::sum (collect (FRE.tanks)) => $result::collect (FRE.tanks) => $result'
                  f'::FRE.Full Mass (p) => $result::FRE.Full Mass::FRE.Full Mass <<Feature>>'] # Result of gathering all instances of Full Mass from the interpretation
sum_1_result = name_to_id_lookup[f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real::+ (sum (collect (FRE.engines)),'
                                 f' sum (collect (FRE.tanks))) => $result::sum (collect (FRE.tanks)) => $result::'
                                 f'sum (collect (FRE.tanks)) <<Feature>>'] # Result of the sum Expression on Full Mass
rt_10_isp_id = name_to_id_lookup[f"{PARTS_LIBRARY}RT-10 \"Hammer\" Solid Fuel Booster::Specific Impulse: "
                                        f"Real <<AttributeUsage>>"]

real = name_to_id_lookup['Model::Real <<DataType>>']

In [ ]:
fre_1_result

In [ ]:
(f'{ROCKET_BUILDING}Liquid Stage::Full Mass: Real::+ (sum (collect (FRE.engines)),'
                  f' sum (collect (FRE.tanks))) => $result::sum (collect (FRE.tanks)) => $result::collect (FRE.tanks) => $result'
                  f'::FRE.Full Mass (p) => $result::FRE.Full Mass::FRE.Full Mass <<Feature>>')

In [ ]:
'Model::Kerbal::Rocket Building::Liquid Stage::Full Mass: Real::+ (sum (collect(FRE.engines)), sum (collect (FRE.tanks))) => $result::sum (collect (FRE.tanks)) => $result::collect (FRE.tanks) => $result::FRE.Full Mass (p) => $result::FRE.Full Mass::FRE.Full Mass <<Feature>>'

## Understanding Expressions

Eventually this will move out to its own notebook ...

Expressions form trees inside the M1 model. Some work has to be done in order to understand them and how data should flow from result parameters in one expression to the input parameters of another.

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [ ]:
exec_order = generate_execution_order(lpg)

In [ ]:
pprint_edges(exec_order, helper_client.model)

In [ ]:
m0_interpretation = random_generator_playbook(
    lpg,
    shorten_pre_bake,
)

In [ ]:
pprint_single_id_list(list(m0_interpretation.keys()), helper_client.model)

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [ ]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.model)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.model.elements[seq[-1]])))

Once the interpretations are generated, we can look for expressions and create an execution order (this is similar to Excel builds a dependency graph internally to accelerate computations and partial updates when a user changes a cell value).

In [ ]:
dcg = generate_execution_order(lpg)

One of the core tools in examining and working with the M1 model is using the get_projection function on the master graph to select out the kind of nodes and edges that will support other queries (roll-up using breadth-first search in reverse order, paths from one node to another to lay out sequences, etc.)

In [ ]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred"), m0_interpretation, dcg)

Display the calculation order determined by the algorithm, as well as a hint about what the step is (generating output from a function, moving values from function parameters to an attribute, applying redefinition, etc.)

In [ ]:
for item in dcg:
    instance_safe = True
    if item[0] not in m0_interpretation or len(m0_interpretation[item[0]]) == 0:
        print("No instances for " + lpg.nodes[item[0]]['qualifiedName'])
        instance_safe = False
    if item[1] not in m0_interpretation or len(m0_interpretation[item[1]]) == 0:
        print("No instances for " + lpg.nodes[item[1]]['qualifiedName'])
        instance_safe = False
    if instance_safe:
        rep_source = m0_interpretation[item[0]][0][-1]
        rep_target = m0_interpretation[item[1]][0][-1]
        if len(item) == 3:
            safe_item = item[2]
        else:
            safe_item = 'None'
        print('(' + str(rep_source) + ', ' + str(rep_target) + ', ' + safe_item + ')')

Use the calculation order in order to resolve the "unset" fields on many attributes to values where they are determined in the M1 model (e.g., using the ' = ' operator to assign values directly or connect to equations or analyses).

In [ ]:
cg.solve_graph(lpg)

In [ ]:
[pse for pse in cg.calculation_log if pse.startswith("[PSE]")]

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

In [ ]:
m0_interpretation[rt_10_isp_id]

In [ ]:
[seq[-1].value for seq in m0_interpretation[rt_10_isp_id]]

In [ ]:
m0_interpretation[ft200_full_mass]

In [ ]:
m0_interpretation[sum_1_result]

In [ ]:
m0_interpretation[x]

In [ ]:
m0_interpretation[liquid_stage_full_mass]

In [ ]:
m0_interpretation[fre_1_result]

In [ ]:
m0_interpretation[real]

Show all interpretation sequence sets (limited to length of 5).

In [ ]:
for print_line in pprint_interpretation(m0_interpretation, lpg.model):
    print(print_line)